# Adults Census Model Training with IBM Watson Machine Learning  
## Scikit-learn: Support Vector Machines Algorithm

## Outline

1. Data Preparation
2. Model Training
3. Model Comparison

In [1]:
# Python libraries that we'll use throughout the analysis
import os
import pandas as pd
import numpy as np

### 2. Data Preparation
We download Adult Census Income Binary classfication dataset from github and preprocessing.

- 2.1 Download dataset

- 2.2 Data preprocessing

Dataset: https://archive.ics.uci.edu/ml/datasets/Adult

#### 2.1 Download Adult Data Set

In [2]:
## read the data
data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", header = None, na_values=' ?')

## read test data
data_test = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test", header = None, skiprows=1, na_values=' ?')

## set column names
data.columns = ['age', 'workclass','fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 
'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'IncomeGroup']

data_test.columns = ['age', 'workclass','fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 
'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'IncomeGroup']

#### 2.2 Data Preprocessing

In [3]:
np.unique(data.loc[:,'IncomeGroup'].values.tolist())

array([' <=50K', ' >50K'], dtype='<U6')

In [4]:
np.unique(data_test.loc[:,'IncomeGroup'].values.tolist())

array([' <=50K.', ' >50K.'], dtype='<U7')

In [5]:
# preprocess

d = {' <=50K': 0,
     ' >50K': 1}
data['IncomeGroup'] = data['IncomeGroup'].map(d)

t = {' <=50K.': 0,
     ' >50K.': 1}
data_test['IncomeGroup'] = data_test['IncomeGroup'].map(t)

# Combine the two datasets to convert the categorical values to binary indicators
data_combined = pd.concat([data, data_test])
c = {0 : ' <=50K',
     1 : ' >50K'}
data_combined['IncomeGroup'] = data_combined['IncomeGroup'].map(c)
print(f"data_combined.shape: {data_combined.shape}")

# drop rows with missing
data_combined = data_combined.dropna()
print(f"data_combined.shape (After dropna): {data_combined.shape}")

# display data
data_combined.head()

data_combined.shape: (48842, 15)
data_combined.shape (After dropna): (45222, 15)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,IncomeGroup
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [6]:
np.unique(data_combined.loc[:,'IncomeGroup'].values.tolist())

array([' <=50K', ' >50K'], dtype='<U6')

In [7]:
# summarize the class distribution
import collections

target = data_combined.values[:,-1]
counter = collections.Counter(target)
for k,v in counter.items():
    per = v / len(target) * 100
    print('Class=%s, Count=%d, Percentage=%.3f%%' % (k, v, per))

Class= <=50K, Count=34014, Percentage=75.216%
Class= >50K, Count=11208, Percentage=24.784%


In [8]:
cat_ix = data_combined.select_dtypes(include=['object', 'bool']).columns
cat_ix

Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country', 'IncomeGroup'],
      dtype='object')

In [9]:
num_ix = data_combined.select_dtypes(include=['int64', 'float64']).columns
num_ix

Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')

### 3 Model Training

In [16]:
#! pip install watermark
%reload_ext watermark
%watermark  -d -u -h -m -a "WML Notebook Instance" -v -p numpy,pandas,scipy,sklearn,ibm_boto3,botocore,ibm_watson_machine_learning

# The runtime has 1 vCPU and 4 GB RAM.

Author: WML Notebook Instance

Last updated: 2020-12-09

Python implementation: CPython
Python version       : 3.7.9
IPython version      : 7.15.0

numpy                      : 1.18.5
pandas                     : 1.0.5
scipy                      : 1.5.0
sklearn                    : 0.23.1
ibm_boto3                  : 2.7.0
botocore                   : 1.16.11
ibm_watson_machine_learning: 1.0.43

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 4.15.0-122-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 56
Architecture: 64bit

Hostname: notebook-conda1py3725f880f6ada9447e8629ea063173a606-54c7956bhrd



In [12]:
# Python libraries 
import time
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import SVC

In [13]:
start_time = time.time()

#Prepocessing
print('building datasets')
features = 'age workclass fnlwgt education education-num marital-status occupation relationship race sex capital-gain capital-loss hours-per-week native-country'
target = 'IncomeGroup'
X = data_combined[features.split()]
y = data_combined[target]

# label encode the target variable to have the classes 0 and 1
y = LabelEncoder().fit_transform(y)

cat_ix = X.select_dtypes(include=['object', 'bool']).columns
print(f"cat_ix: {cat_ix}")
num_ix = X.select_dtypes(include=['int64', 'float64']).columns
print(f"num_ix: {num_ix}")

#use stratify for un balanced number of examples for each class label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42,shuffle=True)
print(f"X_train:{X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train:{y_train.shape}")
print(f"y_test: {y_test.shape}")
  
# Model Training: Support Vector Machines
print('training model')
MODEL = 'SVC'

# define steps
model = SVC(gamma='scale', probability = True, max_iter = -1)
steps = [('c',OneHotEncoder(handle_unknown='ignore'),cat_ix), ('n',MinMaxScaler(),num_ix)]
# one hot encode categorical, normalize numerical
ct = ColumnTransformer(steps)
# wrap the model a pipeline
pipe  = Pipeline(steps=[('transformer',ct),('svc',model)])
pipe.fit(X_train, y_train)
print(f"model: {pipe}")
  
## Evaluate the model

# using threshold
threshold = 0.5 # set threshold as 0.5
y_pred = (pipe.predict_proba(X_test)[:,1] >= threshold).astype(int) 
print(f"y_pred: {y_pred}")
print(f'Misclassified examples:{(y_test != y_pred).sum()}')
print(pipe.predict_proba(X_test))
  
print(f"distinct y_pred: {np.unique(y_pred)}")
print(f"distinct y_test: {np.unique(y_test)}")
  
# Confusion Matrix
conf_mat = metrics.confusion_matrix(y_test, y_pred)
print(f"confusion matrix:\n {conf_mat}")

# AUC
auc = metrics.roc_auc_score(y_test, y_pred)
print(f"auc: {auc}")

# Accuracy
accuracy = metrics.accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}") 

# Precision
precision = metrics.precision_score(y_test, y_pred)
print(f"Precision: {precision}")

# Recall
recall = metrics.recall_score(y_test, y_pred)
print(f"Recall: {recall}")

# F1
f1 = metrics.f1_score(y_test, y_pred)
print(f"F1: {f1}")

# classification_report
print('classification_report')
print(metrics.classification_report(y_test, y_pred))

# persist model
# path = f"models/SageMaker/Adult/local/desktop/{MODEL}.joblib"
# joblib.dump(pipe, path)
# print('model persisted at ' + path)

time_elapsed = (time.time() - start_time)
print("Computational time:{}".format(time_elapsed))

building datasets
cat_ix: Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object')
num_ix: Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')
X_train:(31655, 14)
X_test: (13567, 14)
y_train:(31655,)
y_test: (13567,)
training model
model: Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('c',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object')),
                                                 ('n', MinMaxScaler(),
                                                  Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype=

#### save model

In [15]:
from botocore.client import Config
import ibm_boto3

client = ibm_boto3.client(service_name='s3',
                          ibm_api_key_id='7rULUuPgOFK63b96NYtJ9q1SzM16E4cyWVeUCIg-ukte',
                          ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
                          config=Config(signature_version='oauth'),
endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

# persist model
joblib.dump(pipe, f"{MODEL}.joblib")
client.upload_file(Filename=f"{MODEL}.joblib",
                   Bucket='adultscensusproject-donotdelete-pr-7nekr9yeith2lc',
                   Key=f"{MODEL}.joblib")